# Pivots

## Introduction

Most people likely have experience with pivot tables in Excel. Pandas provides a similar function called (appropriately enough) pivot_table . While it is exceedingly useful, I frequently find myself struggling to remember how to use the syntax to format the output for my needs. This notebook will focus on explaining the pandas `pivot_table` function and how to use it for your data analysis.



## The Data

One of the challenges with using the panda’s pivot_table is making sure you understand your data and what questions you are trying to answer with the pivot table. It is a seemingly simple function but can produce very powerful analysis very quickly.

THis example is going to track a sales pipeline (also called funnel). The basic problem is that some sales cycles are very long (think “enterprise software”, capital equipment, etc.) and users might want to understand it in more detail throughout the year.

Typical questions include:

> 
> How much revenue is in the pipeline? <BR>
> What products are in the pipeline?<BR>
> Who has what products at what stage?<BR>
> How likely are we to close deals by year end?<BR>
>
Many companies will have CRM tools or other software that sales uses to track the process. While they may have useful tools for analyzing the data, inevitably someone will export the data to Excel and use a PivotTable to summarize the data.

Using a panda’s pivot table can be a good alternative because it is:

- Quicker (once it is set up)
- Self documenting (look at the code and you know what it does)
- Easy to use to generate a report or email
- More flexible because you can define custome aggregation functions



## Read in the Data

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(filepath_or_buffer="../Data/sales.csv")
display(df.head())
df.dtypes

## Define a Category

For convenience only, define the `status` column as a category and set the order we want to view.

Not strictly required but helps to keep the order we want as we work through analyzing the data.

Note the change in type for the `Status` column. It was an object and is now a category.

In [ ]:
df["Status"] = df["Status"].astype("category")
df["Status"] = df["Status"].cat.set_categories(["won","pending","presented","declined"])

display(df.head())
df.dtypes

## Pivot the data - single Index

Generally best to pivot one step at a time. Add items and check each step to verify you are getting the results expected. Don’t be afraid to play with the order and the variables to see what presentation makes the most sense for your needs.

The simplest pivot table **must** have a dataframe and an index . In this case, use the `Name` column as the index.

**Note** the only columns in the pivto table are the numeric ones

In [ ]:
vals = ['Price','Quantity']
ndx = 'Rep'

pd.pivot_table(df,index=ndx, values=vals)

In [ ]:
vals = ['Price','Quantity']
ndx = 'Manager'

pd.pivot_table(df,index=ndx, values=vals)

## Pivot the data - Multiple Indices

You can have multiple indexes as well. In fact, most of the pivot_table args can take multiple values via a list.

In [ ]:
vals = ['Price','Quantity']
ndx = ['Manager','Rep','Name']

pd.pivot_table(df,index=ndx, values=vals)

Look at this by Manager and Rep. It’s easy enough to do by changing the index .

In [ ]:
vals = ['Price','Quantity']
ndx = ['Manager','Rep']

pd.pivot_table(df,index=ndx, values=vals)

## Aggrating and Summarizing Data

The pivot table is smart enough to start aggregating the data and summarizing it by grouping the reps with their managers. 

For this purpose, the `Account` and `Quantity` columns aren’t really useful. Let’s remove it by explicitly defining the columns we care about using the values field.

In [ ]:
vals = ['Price']
ndx = ['Manager','Rep']

pd.pivot_table(df,index=ndx,values=vals)

The price column automatically averages the data (calcultes the mean) but we can do a `count` or a `sum`. Adding them is simple using aggfunc and np.sum .


In [ ]:
vals = ['Price']
ndx = ['Manager','Rep']

pd.pivot_table(df,index=ndx,values=vals,aggfunc=np.sum)

The `aggfunc` parameter can take a list of functions. E.g. using the numpy `mean` function and `len` to get a count.

In [ ]:
vals = ['Price']
ndx = ['Manager','Rep']
funcs = [np.mean,len]

pd.pivot_table(df,index=ndx,values=vals,aggfunc=funcs)

The `aggfunc` parameter can take a list of names of functions. <BR>
E.g. using the numpy functions np.sum, np.mean and np.std

In [ ]:
vals = ['Price']
ndx = ['Manager','Rep']
funcs = [np.sum, np.mean, np.std]

pd.pivot_table(df,index=ndx,values=vals,aggfunc=funcs)

If we want to see sales broken down by the products, the `columns` paramneter allows you to define one or more columns.

## Columns vs. Values
One of the confusing points with the pivot_table is the use of columns and values .  
Remember, columns are optional - they provide an additional way to segment the actual values you care about.  
The aggregation functions are applied to the **values** you list.

In [ ]:
vals = ['Price']
ndx = ['Manager','Rep']
cols = ['Product']

pd.pivot_table(df,index=ndx,values=vals, columns=cols,aggfunc=[np.sum])

Remove the Nans bu using the `fill_value` to set them to 0.

In [ ]:
vals = ['Price']
ndx = ['Manager','Rep']
cols = ['Product']

pd.pivot_table(df,index=ndx, values=vals, columns=cols, aggfunc=[np.sum], fill_value=0)

Add Quantity to the values list.

In [ ]:
vals = ["Price","Quantity"]
ndx = ['Manager','Rep']
cols = ['Product']

pd.pivot_table(df,index=ndx, values=["Price","Quantity"], columns=["Product"], aggfunc=[np.sum], fill_value=0)

## Move items to the index 

This gives a different visual representation. 

Remove Product from the columns and add to the index .

In [ ]:
vals = ["Price","Quantity"]
ndx = ["Manager","Rep","Product"]
cols = ['Product']

pd.pivot_table(df,index=ndx, values=vals, aggfunc=[np.sum], fill_value=0)

## Add some totals

Set the `margins` parameter to **True** .

The totals are in the last column `All`

In [ ]:
vals = ["Price","Quantity"]
ndx = ["Manager","Rep","Product"]
funcs = [np.sum,np.mean]

pd.pivot_table(df,index=ndx, values=vals, aggfunc=funcs, fill_value=0, margins=True)

## Different Functions for Different Values

If you wenat to have different aggregate functions to different values, use a dictionary

For example
> **len** for the `Quuantity` value <BR>
> **np.sum** for the `Price` column <BR>

In [ ]:
vals = ["Price","Quantity"]
ndx = ["Manager","Status"]
cols=["Product"]

funcs = {
    "Quantity":len,
    "Price":np.sum
}

pd.pivot_table(df, index=ndx, columns=cols, values=vals, aggfunc=funcs, fill_value=0)


It is also possible to provide a list of aggfunctions to apply to each value too:

For example <BR>
> **len** for `Quantity` values  <BR>
> **np.sum** and **np.mean** for the `Price` value  <BR>



In [ ]:
vals = ["Price","Quantity"]
ndx = ["Manager","Status"]
cols=["Product"]


funcs = {
    "Quantity":len,
    "Price":[np.sum,np.mean]
}

table = pd.pivot_table(df,index=ndx, columns=cols, values=vals, aggfunc=funcs, fill_value=0)
table

## Advanced Pivot Table Filtering

Once you have generated your data, it is in a DataFrame so you can filter on it using your standard DataFrame functions.

This uses ther `query` method on a DataFrame

For example - filter for the manager equal to Debra Hanley

In [ ]:
is_mgr_Deb_Han = 'Manager == ["Debra Henley"]'

table.query(is_mgr_Deb_Han)

All of our pending and won deals.

In [ ]:
pending_or_won = 'Status == ["pending","won"]'

table.query(pending_or_won)

## Add some colour

### High Low colours

In [ ]:
vals = ["Price","Quantity"]
ndx = ["Manager","Status"]
cols=["Product"]


funcs = {
    "Quantity":len,
    "Price":[np.sum,np.mean]
}

table = pd.pivot_table(df, index=ndx, columns=cols, values=vals, aggfunc=funcs, fill_value=0)

table.style.highlight_max(color='lightgreen').highlight_min(color='red')

### Gradient Colours

In [ ]:
table.style.background_gradient(cmap='Blues')

### Gradient Colours

Similar to `numpy.where`

Used to highlight based on a condition


In [ ]:
def style_func(v, value, other):
    cond = v > 65000 
    return value if cond else other

table.style.applymap(style_func, value='color:green;', other='color:red;')


# Cheat - Sheet

<img style="float: center;" width="720" src="../Images/pivot-table.png">